<h1><center>Stablecoin Billionaires<br> Descriptive Analysis of the Ethereum-based <br>
    Stablecoin ecosystem</center></h1>  

<h2><center> by Anton Wahrstätter<br>01.07.2020</h2></center>

<h2><center>Part III - Paxos Standard</h2></center>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from matplotlib import rc
import re
import random

rc('font', **{'family':'serif','serif': ['Computer Modern']})
rc('text', usetex=True)

#decimals
dec = 18

In [2]:
#plots
tx_over_date = '../plots/paxos/paxos_txs_over_date.csv'
unique_senders_over_date = '../plots/paxos/paxos_unique_senders_over_date.csv'
unique_recipients_over_date = '../plots/paxos/paxos_unique_recipients_over_date.csv'
tx_count_to = '../plots/paxos/paxos_tx_count_to.csv'
tx_count_from = '../plots/paxos/paxos_tx_count_from.csv'
tx_over_date = '../plots/paxos/paxos_txs_over_date.csv'
balances = '../plots/paxos/paxos_balances.csv'
avg_gas_over_date = '../plots/paxos/paxos_avg_gas_over_date.csv'
avg_value_over_date = '../plots/paxos/paxos_avg_value_over_date.csv'
positive_cumulated_balances = '../plots/paxos/paxos_positive_cumulated_balances.csv'
circulating_supply = '../plots/paxos/paxos_circulating_supply.csv'
unique_recipients_per_day_over_date = '../plots/paxos/paxos_unique_recipients_per_day_over_date.csv'
unique_senders_per_day_over_date = '../plots/paxos/paxos_unique_senders_per_day_over_date.csv'
exchanges = '../plots/exchanges.csv'

#data 
transfer = '../data/paxos/transfer/0_paxos_transfer_6294931-10370273.csv'
mint = '../data/paxos/supplyincreased/paxos_supplyincreased.csv'
burn = '../data/paxos/supplydecreased/paxos_supplydecreased.csv'

<center></center>

# Data

In [3]:
df = pd.read_csv(transfer)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Basics

In [4]:
df['txvalue'] = df['txvalue'].astype(float)/10**18

In [5]:
df.describe()

,timestamp,blocknumber,txindex,txvalue,gas_price,gas_used
count,2580722.000,2580722.000,2580722.000,2580722.000,2580722.000,2580722.000
mean,1584005181.464,9665616.218,78.687,8879.987,15768757613.356,152639.472
std,9869912.885,714086.655,49.558,162386.313,14983724913.008,128012.264
min,1536584636.000,6306166.000,0.000,0.000,0.000,24375.000
25%,1582370892.000,9532888.500,40.000,20.000,5000000000.000,54823.000
50%,1587413738.000,9911407.000,71.000,100.000,11300000000.000,104887.000
75%,1590178871.000,10117748.000,110.000,500.000,23000000000.000,197075.000
max,1593561562.000,10370272.000,491.000,57852545.303,1500000000000.000,5182695.000


<center></center>

## Dataset

In [6]:
print('Start:')
print('Block: {:^30}\nTimestamp: {:^20}\nUTC Time: {:^25}\n'.format(df['blocknumber'].iloc[0],
                                                        df['timestamp'].iloc[0],
                                                        str(datetime.fromtimestamp(df['timestamp'].iloc[0]))
                                                       ))
print('End:')
print('Block: {:^30}\nTimestamp: {:^20}\nUTC Time: {:^25}\n'.format(df['blocknumber'].iloc[-1],
                                                        df['timestamp'].iloc[-1],
                                                        str(datetime.fromtimestamp(df['timestamp'].iloc[-1]))
                                                       ))

Start:
Block:            6306166            
Timestamp:      1536584636     
UTC Time:    2018-09-10 15:03:56   

End:
Block:            10370272           
Timestamp:      1593561562     
UTC Time:    2020-07-01 01:59:22   



## Total Nr. of Blocks

In [7]:
print('Total Nr. of Blocks: {}'.format(df['blocknumber'].iloc[-1]-df['blocknumber'].iloc[0]))

Total Nr. of Blocks: 4064106


<center></center>

## Total Nr. of Transfer Events

In [8]:
print('Total Nr. of Events: {:,.0f}'.format(df.describe().loc['count','timestamp']))

Total Nr. of Events: 2,580,722


<center></center>

## Total Nr. of Addresses

In [9]:
print('Total Nr. of Addresses: {}'.format(len(df['txto'].unique())))

Total Nr. of Addresses: 1029956


<center></center>

## Addresses with funds

In [10]:
bal = pd.read_csv(balances)
print('Total Nr. of Addresses with funds: {}'.format(len(bal[bal['txvalue']>0])))

Total Nr. of Addresses with funds: 86004


<center></center>

## Avg. Transaction Value

In [11]:
print('Avg. Transaction Value: {:,.0f} Pax'.format(np.mean(df['txvalue']/10**dec)))

Avg. Transaction Value: 8,880 Pax


<center></center>

## Total Gas Costs

In [12]:
df['costs'] = (df['gas_price']/10**dec) * df['gas_used']
print('Total Gas spent for Transfers: {:,.3f} ether'.format(sum(df['costs'])))

Total Gas spent for Transfers: 5,558.547 ether


<center></center>

## Total Paxos Supply

In [13]:
(sum(pd.read_csv(mint)['txvalue'].astype(float))-sum(pd.read_csv(burn)['txvalue'].astype(float)))/(10**dec)

241123246.6999984

<center></center>

In [14]:
#other method
sum(df[df['txfrom'] == '0x0000000000000000000000000000000000000000']["txvalue"].astype(float))/(10**dec)- sum(df[df['txto'] == '0x0000000000000000000000000000000000000000']["txvalue"].astype(float))/(10**dec)

241123246.69999838

<center></center>

<center></center>

<center></center>

# I. Event analysis

## I.I. Mint Event

## Plot new issued tokens over date

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(40,25), dpi=250)
ax = fig.subplots()
plt.grid()
plt.title(r'I s s u e d \ \ P A X'+'\n', size= 120)
ax.yaxis.get_offset_text().set_fontsize(50)
plt.xlabel('\n'+r'D a t e ', size=120)
plt.ylabel(r'P A X'+'\n', size=120)
plt.yticks(fontsize=60)
plt.xticks(labels=["Okt '18","\nJan '19",
                   "Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[21,113,
                  203,294,386,
                  478,569,660], fontsize=60)

def plot_issue_over_date():
    _issue = pd.read_csv(mint)
    _issue["txvalue"] = _issue["txvalue"].astype(float)
    iss = _issue.loc[:, ['timestamp', 'txvalue']]
    iss['utc'] = iss['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[0:10])
    iss = iss.groupby('utc', as_index = False)['txvalue'].sum()
    a = iss['utc'].iloc[0]
    b = iss['utc'].iloc[-1]
    idx = pd.date_range(a,b)
    iss = iss.set_index('utc')
    iss.index = pd.DatetimeIndex(iss.index)
    iss = iss.reindex(idx, fill_value=0)
    counter = 0
    for i in range(0, len(iss)):
        plt.plot([counter,counter], [0, iss['txvalue'].iloc[counter]/(10**dec)], color= 'black', linewidth=3)
        counter += 1
    return 

plt.tight_layout(pad=5)
plot_issue_over_date()
plt.savefig('../pics/paxos/paxos_issued_pax_over_date.pdf')

## Further info

In [16]:
df = pd.read_csv(mint)
df['txvalue'] = df['txvalue'].astype(float)
#print(df[df['txvalue'] == max(df['txvalue'])])
print('Issue Events: {}\nIssued Paxos: {:,.0f}\n'.format(len(df), sum(df['txvalue'])/10**dec, ':,0f'))
print('Largest issue: {:,.0f} Paxos\n . . . to address: {}\n'.format(df.loc[716, 'txvalue']//10**dec,'0x55fe002aeff02f77364de339a1292923a15844b8'))

Issue Events: 940
Issued Paxos: 1,733,264,062

Largest issue: 21,912,463 Paxos
 . . . to address: 0x55fe002aeff02f77364de339a1292923a15844b8



<center></center>

<center></center>

## I.II. Burn Event

## Plot burned tokens over date

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(40,25))
ax = fig.subplots()
plt.grid()
plt.title(r'B u r n e d \ \ P A X'+'\n', size= 120)
ax.yaxis.get_offset_text().set_fontsize(50)
plt.xlabel('\n'+r'D a t e', size=120)
plt.ylabel(r'P A X'+'\n', size=120)
plt.yticks(fontsize=60)
plt.xticks(labels=["Okt '18","\nJan '19",
                   "Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[19,111,
                  201,292,384,
                  476,567,658], fontsize=60)

def plot_burn_over_date():
    _dbf = pd.read_csv(burn)
    _dbf["txvalue"] = _dbf["txvalue"].astype(float)
    dbf = _dbf.loc[:, ['timestamp', 'txvalue']]
    dbf['utc'] = dbf['timestamp'].apply(lambda x: str(datetime.utcfromtimestamp(x))[0:10])
    dbf = dbf.groupby('utc', as_index = False)['txvalue'].sum()
    a = dbf['utc'].iloc[0]
    b = dbf['utc'].iloc[-1]
    idx = pd.date_range(a,b)
    dbf = dbf.set_index('utc')
    dbf.index = pd.DatetimeIndex(dbf.index)
    dbf = dbf.reindex(idx, fill_value=0)
    counter = 0
    for i in range(0, len(dbf)):
        plt.plot([counter,counter], [0, dbf['txvalue'].iloc[counter]/(10**dec)], color= 'black', linewidth=3)
        counter += 1
    
    return

plt.tight_layout(pad=5)
plot_burn_over_date()
plt.savefig('../pics/paxos/paxos_burned_pax_over_date.pdf')

## Further info

In [18]:
df = pd.read_csv(burn)
df['txvalue'] = df['txvalue'].astype(float)
idx = df[df['txvalue'] == max(df['txvalue'])].index[0]
df['txvalue'].iloc[idx]

print('Burn Events: {}\nBurned pax: {:,.0f}'.format(len(df), sum(df['txvalue'])/10**dec, ':,0f'))
print('. . . from {} addesses\n'.format(len(df['address'].unique())))
print('Largest burn: {:,.0f} pax\n . . . from address: {}\n'.format(df['txvalue'].iloc[idx]/(10**dec), 
                                                                     df.groupby("address")["txvalue"].sum().index[0]))

Burn Events: 906
Burned pax: 1,492,140,815
. . . from 2 addesses

Largest burn: 25,299,564 pax
 . . . from address: 0x5195427ca88df768c298721da791b93ad11eca65



<center></center>

<center></center>

## Plot circulating supply

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(20,12), dpi=500)
ax = fig.subplots()
plt.grid(True)
plt.title(r'C i r c u l a t i n g \ \ P A X \ \ S u p p l y'+'\n', size=60)
plt.xlabel('\n'+r'D a t e', size= 60)
plt.ylabel(r'P A X'+'\n', size= 60)
ax.yaxis.get_offset_text().set_fontsize(25)

plt.yticks(fontsize=30)
plt.xticks(labels=["Okt '18","\nJan '19",
                   "Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[21,113,
                  203,294,386,
                  478,569,660], fontsize=30)

circ = pd.read_csv(circulating_supply, index_col='Unnamed: 0')

plt.plot(range(0, 660), circ['txvalue'].cumsum()/10**dec, color='black', linewidth = 4, label = 'Paxos supply')
plt.fill_between(range(0, 660),0 , circ['txvalue'].cumsum()/10**dec, alpha=0.2, facecolor='#2D728F')
lgnd = plt.legend(loc='upper left', fontsize=40)
plt.tight_layout(pad=5)
plt.savefig('../pics/paxos/paxos_cirulating_supply.pdf')

<center></center>

<center></center>

<center></center>

## I.III. Transfer Event

## Plot transfers over date

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(20,12), dpi=500)
ax = fig.subplots()
plt.grid(True)
plt.title(r'P A X \ \ T r a n s f e r s'+'\n', size=60)
plt.xlabel('\n'+r'D a t e', size= 50)
plt.ylabel(r'T r a n s f e r s'+'\n', size= 50)
plt.yticks(np.arange(0, 30001, 5000), 
           np.vectorize(lambda x: f'{x:,.0f}')(np.arange(0, 30001, 5000)), 
           fontsize=30)
plt.xticks(labels=["Okt '18","\nJan '19",
                   "Apr '19","\nJul '19","Oct '19",
                   "\nJan '20","Apr '20","\nJul '20"], 
           ticks=[21,113,
                  203,294,386,
                  478,569,660], fontsize=30)

def plot_txs_over_date(df, lwd, label, col = '#2D728F', fillbetween=False):
    plt.plot(np.arange(0 , len(df['txs'])), df['txs'], color = col, linewidth = lwd, label = label)
    if fillbetween:
        plt.fill_between(np.arange(0 , len(df['txs'])),0 , df['txs'], alpha=0.2, facecolor='#2D728F')    
plot_txs_over_date(df = pd.read_csv(tx_over_date, index_col=0), 
                   col = 'black', lwd = 2, label = 'Transfers', fillbetween=True)

plot_txs_over_date(pd.read_csv(unique_recipients_per_day_over_date, index_col='Unnamed: 0'), 
                   lwd = 2, label = 'Unique Recipients per day')

plot_txs_over_date(pd.read_csv(unique_senders_per_day_over_date, index_col='Unnamed: 0'), 
                   col='#9DB469', lwd = 2, label =  'Unique Senders per day')

lgnd = ax.legend(loc='upper left', fontsize=35)
lgnd.legendHandles[0].set_linewidth(5.0)
lgnd.legendHandles[1].set_linewidth(5.0)
lgnd.legendHandles[2].set_linewidth(5.0)

plt.tight_layout(pad=5)
plt.savefig('../pics/paxos/paxos_tx_over_date.pdf')
plt.show()

<center></center>

## Most active addresses

From:

In [21]:
fr = pd.read_csv(tx_count_from, index_col='Unnamed: 0').sort_values('txs', ascending = False)
to = pd.read_csv(tx_count_to, index_col='Unnamed: 0').sort_values('txs', ascending = False)
fr = pd.DataFrame(fr.loc[:fr.index[10],'txs'])
fr['tag'] = ['MMM BSC', 'Scammer', 'Kyber: Contract', '-', 'Binance', '-', '-', 'Binance 4 ', 'Binance 3', 'Binance 2', 'BillionMoney.live']
fr

,txs,tag
0x8a91c9a16cd62693649d80afa85a09dbbdcb8508,308538,MMM BSC
0xc88f7666330b4b511358b7742dc2a3234710e7b1,78751,Scammer
0x65bf64ff5f51272f729bdcd7acfb00677ced86cd,31926,Kyber: Contract
0x41f8d14c9475444f30a80431c68cf24dc9a8369a,30765,-
0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,15835,Binance
0x6f7b74aa23cabd638d7ebe804577043c435192b9,15054,-
0xc5a8859c44ac8aa2169afacf45b87c08593bec10,14816,-
0x0681d8db095565fe8a346fa0277bffde9c0edbbf,11561,Binance 4
0x564286362092d8e7936f0549571a803b203aaced,11435,Binance 3
0xd551234ae421e3bcba99a0da6d736074f22192ff,11345,Binance 2


To:

In [22]:
to = pd.DataFrame(to.loc[:to.index[10],'txs'])
to['tag'] = ['MMM BSC', '-', 'Scammer', 'Binance', '-', 'Kyber: Contract', '-', 'BillionMoney.live', '-', '-', '-']
to

,txs,tag
0x8a91c9a16cd62693649d80afa85a09dbbdcb8508,341716,MMM BSC
0x6f7b74aa23cabd638d7ebe804577043c435192b9,162555,-
0xc88f7666330b4b511358b7742dc2a3234710e7b1,56733,Scammer
0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,35865,Binance
0x41f8d14c9475444f30a80431c68cf24dc9a8369a,30763,-
0x65bf64ff5f51272f729bdcd7acfb00677ced86cd,30126,Kyber: Contract
0x5195427ca88df768c298721da791b93ad11eca65,15997,-
0xff8e926d0d92b5da930f5534a79e5b821f719f8a,10997,BillionMoney.live
0x7cd860672c477e4c312cdde6f922c51235caf52d,10123,-
0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,8045,-


<center></center>

<center></center>

## Activity distribution

In [23]:
df_from = pd.read_csv(tx_count_from, index_col=0)
df_to = pd.read_csv(tx_count_to, index_col=0)
df_all = pd.concat([df_from, df_to])
df = df_all.groupby(df_all.index).sum()
print('{} addresses in total'.format(len(df)))
df = df.sort_values('txs')
gr0 = len(df.loc[df['txs'] >= 500000])
gra = len(df.loc[df['txs'] >= 100000]) - gr0 
grb = len(df.loc[df['txs'] >= 50000]) - gr0 - gra
grc = len(df.loc[df['txs'] >= 10000]) - gr0 - gra - grb
grd = len(df.loc[df['txs'] >= 1000]) - gr0 - gra - grb - grc
gre = len(df.loc[df['txs'] >= 100]) - gr0 - gra - grb - grc - grd
grf = len(df.loc[df['txs'] >= 10]) - gr0 - gra - grb - grc - grd - gre
grg = len(df.loc[df['txs'] <= 10])
grh = len(df.loc[df['txs'] == 1]) 
pd.DataFrame({'Transactions': ['> 500.000','100.000-500.000',
                               '50.000-100.000',
                               '10.000-50.000',
                               '1.000-10.000',
                               '100-1.000',
                               '10-100',
                               '< 10', 
                               '1'], 
              'Addresses':[gr0,gra,grb,grc,grd,gre,grf,grg,grh]
             })

1029973 addresses in total


,Transactions,Addresses
0,> 500.000,1
1,100.000-500.000,2
2,50.000-100.000,3
3,10.000-50.000,11
4,1.000-10.000,125
5,100-1.000,2468
6,10-100,33112
7,< 10,996931
8,1,23107


<center></center>

<center></center>

## Plot average transfer amount

## Jan '20 - Jul '20

In [ ]:
print('\n\n')
df = pd.read_csv(avg_value_over_date, index_col=0)
df = df.loc[df.index[478]:,:]
plt.figure(figsize=(12, 7), dpi=800)

plt.grid(True)
plt.plot(np.arange(0 , len(df.index.tolist())), df['txvalue'], color = 'black', label = 'Avg. Amount/Day', linewidth = 2)
plt.fill_between(np.arange(0 , len(df.index.tolist())),0 , df['txvalue'], alpha=0.2, facecolor='#2D728F')
plt.xlabel('\n'+'D a t e', fontsize=35)
plt.ylabel('P A X'+'\n', fontsize=30)
plt.title("P a x o s \ \ S t a n d a r d\nA v g. \ \ T r a n s f e r \ \ A m o u n t"+"\n", size = 30)
plt.legend(loc="upper right", fontsize=20, shadow= True)
plt.ticklabel_format(style = 'plain')
plt.xticks(labels=["\nJan '20","Feb '20","\nMar '20","Apr '20","\nMay '20","Jun '20","\nJul '20"], 
           ticks=[0,31,60,90,121,152,182], fontsize=23) 
plt.yticks(np.arange(0, 50001, 10000), 
           np.vectorize(lambda x: f'{x:,.0f}')(np.arange(0, 50001, 10000)), 
           fontsize=15)
plt.tight_layout(pad=1)
plt.savefig('../pics/paxos/paxos_avgtxvalue_jan20.pdf')

<center></center>

## Further Info

In [25]:
df.describe()

,txvalue
count,182.000
mean,5376.390
std,5658.326
min,477.272
25%,2005.187
50%,3695.525
75%,6202.192
max,42037.668


<center></center>

<center></center>

## Plot average gas costs

## Jan '20 - Jul '20

In [ ]:
print('\n\n')
df = pd.read_csv(avg_gas_over_date)
df = df.loc[df.index[478]:,:]
plt.figure(figsize=(12, 7), dpi=800)
plt.grid(True)
plt.plot(np.arange(0 , len(df.index.tolist())), df['gas'], color = 'black', label = 'Avg. Gas Costs/Day', linewidth = 2)
plt.fill_between(np.arange(0 , len(df.index.tolist())),0 , df['gas'], alpha=0.2, facecolor='#2D728F')
plt.xlabel('\nD a t e', fontsize=35)
plt.ylabel('E t h e r\n', fontsize=30)
plt.title("P a x o s \ \ S t a n d a r d\nA v g. \ \ T r a n s f e r \ \ A m o u n t"+"\n", size = 30)
lgnd = plt.legend(loc='upper left', fontsize=20, shadow= True)
plt.ticklabel_format(style = 'plain')
plt.xticks(labels=["\nJan '20","Feb '20","\nMar '20","Apr '20","\nMay '20","Jun '20","\nJul '20"], 
           ticks=[0,31,60,90,121,152,182], fontsize=20) 
plt.yticks(np.arange(0, 0.05, 0.01), 
           np.vectorize(lambda x: f'{x:,.3f}')(np.arange(0, 0.05, 0.01)), 
           fontsize=15)
plt.tight_layout(pad=1)
lgnd.legendHandles[0].set_linewidth(3.0)
plt.savefig('../pics/paxos/paxos_avggascosts_jan20.pdf') 

In [27]:
df.describe()

,gas
count,182.000
mean,0.002
std,0.002
min,0.000
25%,0.001
50%,0.001
75%,0.003
max,0.006


<center></center>

<center></center>

# II. Balances Analysis

In [28]:
df = pd.read_csv(positive_cumulated_balances, index_col='Unnamed: 0')
df

,address,balance,cum
0,0xee28d3003b34e011e8390ef8bb6e2c8cf0c1f444,0.000,0.000
1,0xfa9a04678e55f8e6c99379dded3329852534d3c2,0.000,0.000
2,0x5143fd9259cf7c752f87964d8814be9c49f3fe52,0.000,0.000
3,0x0b5b79ec6aca02f3035be09787cf41e6f40abae1,0.000,0.000
4,0x02a0f7692487de621ef9433de15103f5a88c178b,0.000,0.000
...,...,...,...
85999,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,5940703.355,105825400.235
86000,0x0a1bc490eec8cdf57df1b1a3f71e82e1a057be61,7056447.640,112881847.875
86001,0xc295485f479a45287689a1c27f7cc08d365ce938,7300000.000,120181847.875
86002,0xf977814e90da44bfa03b6295a0616a897441acec,17300138.399,137481986.274


<center></center>

## II.I. Quick Summary

In [29]:
(df[df['balance']>0]['balance']).describe().apply(lambda x: format(x, 'f'))

count        86004.000000
mean          2803.628281
std         363226.153829
min              0.000000
25%              0.126044
50%              4.690000
75%             45.105434
max      103641260.425813
Name: balance, dtype: object

In [30]:
print('{}/{} with less than 1 pax' .format(len(df[df['balance']<1]['balance']), len(df['balance'])))

33037/86004 with less than 1 pax


<center></center>

## II.II. Balance Table

In [31]:
df = pd.read_csv(positive_cumulated_balances, index_col=0)
def get_distribution(perc):
    per = round(df.index[-1]*perc)
    entities = df.index[-1]- per
    upper = df.loc[per:,:]
    lower = df.loc[:per,:]
    lower_ = lower['cum'].iloc[-1]
    upper_ = (upper['cum'].iloc[-1] - upper['cum'].iloc[0])
    return entities, lower_, upper_, lower_/ upper['cum'].iloc[-1], upper_/(upper['cum'].iloc[-1])

idx90, lower90, upper90, per10, per90 = get_distribution(0.90)
idx95, lower95, upper95, per05, per95 = get_distribution(0.95)
idx99, lower99, upper99, per01, per99 = get_distribution(0.99)
idx999, lower999, upper999, per001, per999 = get_distribution(0.999)

df = pd.DataFrame([[f'{idx999:,.0f}', round(per999*100,2), f'{upper999:,.0f}'],
             [f'{idx99:,.0f}', round(per99*100,2),f'{upper99:,.0f}'],
             [f'{idx95:,.0f}', round(per95*100,2),f'{upper95:,.0f}'],
             [f'{idx90:,.0f}', round(per90*100,2),f'{upper90:,.0f}']],
                  index=['0.1% of the richest accounts', '1% of the richest accounts','5% of the richest accounts','10% of the richest accounts'],
                   columns=['Accounts in total', '% of total supply', 'pax amount'])
df

,Accounts in total,% of total supply,pax amount
0.1% of the richest accounts,86,85.080,"205,150,277"
1% of the richest accounts,860,94.640,"228,191,539"
5% of the richest accounts,"4,300",98.240,"236,874,691"
10% of the richest accounts,"8,600",99.230,"239,274,326"


<center></center>

<center></center>

## II.III. Rich list

In [32]:
pd.options.mode.chained_assignment = None
df = pd.read_csv(positive_cumulated_balances)
balance = df
rich = df.loc[df.index[-10]:,:]
ex = pd.read_csv(exchanges, header=None)
loop = rich.iterrows()
for i, j in loop:
    if j['address'] in ex[0].tolist():
        rich.loc[i,'nametag'] = ex[ex[0] == j['address']][1].values[0]


rich.loc[86003,'nametag'] = 'Paxos'
rich.loc[85997,'nametag'] = 'MMM BSC'

rich

,Unnamed: 0,address,balance,cum,nametag
85994,85994,0x5c985e89dde482efe97ea9f1950ad149eb73829b,4375407.001,78322708.508,Huobi 5
85995,85995,0x611479ce1d3d457ffe9fb2abbaf8d3fe2ad8fffb,5000075.001,83322783.509,NaN
85996,85996,0x2a549b4af9ec39b03142da6dc32221fc390b5533,5145839.147,88468622.656,NaN
85997,85997,0x8a91c9a16cd62693649d80afa85a09dbbdcb8508,5572046.493,94040669.150,MMM BSC
85998,85998,0xc5a8859c44ac8aa2169afacf45b87c08593bec10,5844027.730,99884696.880,NaN
85999,85999,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,5940703.355,105825400.235,Binance
86000,86000,0x0a1bc490eec8cdf57df1b1a3f71e82e1a057be61,7056447.640,112881847.875,NaN
86001,86001,0xc295485f479a45287689a1c27f7cc08d365ce938,7300000.000,120181847.875,NaN
86002,86002,0xf977814e90da44bfa03b6295a0616a897441acec,17300138.399,137481986.274,Binance 8
86003,86003,0xe62b5b82fb63f19579f35e44cf998d4819b0322a,103641260.426,241123246.700,Paxos


<center></center>

<center></center>

## Huobi

In [33]:
ex = pd.read_csv(exchanges, header=None)

In [34]:
df = ex.loc[0:73,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Huobi Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal, bal/balance.loc[balance.index[-1], 'cum']*100))

Huobi Total Balance: 5917167
2.45% of Total


<center></center>

## Binance

In [35]:
df = ex.loc[74:88,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Binance Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal, bal/balance.loc[balance.index[-1], 'cum']*100))

Binance Total Balance: 26766779
11.10% of Total


<center></center>

## Bitfinex

In [36]:
df = ex.loc[89:110,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Bitfinex Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal, bal/balance.loc[balance.index[-1], 'cum']*100))

Bitfinex Total Balance: 76526
0.03% of Total


<center></center>

## OKEx

In [37]:
df = ex.loc[111:115,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('OKEx Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal, bal/balance.loc[balance.index[-1], 'cum']*100))

OKEx Total Balance: 1591858
0.66% of Total


<center></center>

## Bittrex

In [38]:
df = ex.loc[116:119,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Bittrex Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal, bal/balance.loc[balance.index[-1], 'cum']*100))

Bittrex Total Balance: 211346
0.09% of Total


<center></center>

## Compound

In [39]:
df = ex.loc[151:179,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Compound Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal, bal/balance.loc[balance.index[-1], 'cum']*100))

Compound Total Balance: 0
0.00% of Total


<center></center>

## Poloniex

In [40]:
df = ex.loc[247:266,:]
bal = 0 
for i in df[0]:
    val = balance['balance'][balance['address'] == i]
    if not val.empty:
        bal += balance['balance'][balance['address'] == i].values[0]
    else:
        pass
print('Poloniex Total Balance: {:.0f}\n{:.2f}% of Total'.format(bal, bal/balance.loc[balance.index[-1], 'cum']*100))

Poloniex Total Balance: 44940
0.02% of Total


<center></center>

<center></center>

<center></center>

# II. IV. Pie Chart

In [41]:
df = pd.read_csv(positive_cumulated_balances, index_col='Unnamed: 0')
aa = df.iloc[df.index[-1]-80:]
bb = df['balance'].iloc[:df.index[-1]-80]
df = aa.append(pd.DataFrame({'address': 'others', 'balance': sum(bb)}, index=[0]))
label = []
counter = 0
def getlabel(i):
    global counter
    if i:
        if not i == 'others':
            label.append(i + '...')
        else:
            label.append(i)
    else:
        label.append('')
    counter += 1
    
[getlabel(i[:6]) if counter >= len(df)-11 else getlabel('') for i in df['address']  ]
print()

In [ ]:
print('\n\n')
# Colorspace colors by: https://colorspace.r-forge.r-project.org/index.html

colorspace_set3 = ['#EEBD92','#FFB3B5','#85D0F2','#BCC3FE','#E7B5F5', 
                 '#FEAFDA', '#61D8D6','#76D9B1','#A4D390','#CFC982']
colorsp_dynamic =['#DB9D85', '#87AEDF', '#9DB469', '#6DBC86', '#3DBEAB', 
                  '#4CB9CC', '#C2A968', '#BB9FE0', '#DA95CC', '#E494AB']
colorspa_dark_3 = ['#B675E0','#5991E4','#00AA5A','#6F9F00','#CE7D3B']
colorspa_dyna_5 = ['#9DB469','#87AEDF','#DA95CC', '#DB9D85','#3DBEAB']

fig = plt.figure(figsize=(25,15), dpi=400)
ax = fig.add_subplot()
aa = plt.pie(df['balance'],colors=colorsp_dynamic, 
             labels=label,
             autopct=lambda x: r'{:.1f}\%'.format(x) if x > 1.5 else r'{:.0f}\%'.format(x) if x > 5  else '', 
             pctdistance= 0.8, 
             labeldistance= 1.05, 
             radius=1, 
             explode=[0.05 for i in range(0, len(df['balance']))],
             wedgeprops = {'linewidth': 0.8, 'edgecolor':'k'}, 
             startangle=250) 

# Custom Modifications
#aa[-1][-1].set_x(-0.7268917458682129)
aa[-1][-1].set_fontsize(35)
aa[-1][-2].set_fontsize(30)
#aa[-1][-2].set_x(0.19977073082370535)
#aa[-1][-2].set_y(0.8000006952023211)
aa[-1][-3].set_fontsize(27)
aa[-1][-4].set_fontsize(23)
aa[-1][-5].set_fontsize(20)
aa[-1][-6].set_fontsize(16)
aa[-1][-7].set_fontsize(13)
aa[-1][-8].set_fontsize(9)
aa[-1][-9].set_fontsize(9)
aa[-1][-10].set_fontsize(9)
aa[-1][-11].set_fontsize(8)

fontsize = -43
for i in aa[1]:
    i.set_fontsize(fontsize)
    fontsize += 1
aa[1][-1].set_fontsize(55)
plt.tight_layout(pad=5)
plt.title('P A X \ \ D i s t r i b u t i o n', fontsize = 50)
circ = plt.Circle((0,0),0.5,color='black', fc='white',linewidth=1.25)
ax.add_artist(circ)
plt.savefig('../pics/paxos/paxos_distribution_pie.pdf')

## II.V. Lorenz curve

In [43]:
df = pd.read_csv(positive_cumulated_balances, index_col = 'Unnamed: 0')
df

,address,balance,cum
0,0xee28d3003b34e011e8390ef8bb6e2c8cf0c1f444,0.000,0.000
1,0xfa9a04678e55f8e6c99379dded3329852534d3c2,0.000,0.000
2,0x5143fd9259cf7c752f87964d8814be9c49f3fe52,0.000,0.000
3,0x0b5b79ec6aca02f3035be09787cf41e6f40abae1,0.000,0.000
4,0x02a0f7692487de621ef9433de15103f5a88c178b,0.000,0.000
...,...,...,...
85999,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,5940703.355,105825400.235
86000,0x0a1bc490eec8cdf57df1b1a3f71e82e1a057be61,7056447.640,112881847.875
86001,0xc295485f479a45287689a1c27f7cc08d365ce938,7300000.000,120181847.875
86002,0xf977814e90da44bfa03b6295a0616a897441acec,17300138.399,137481986.274


In [44]:
y_all = df['cum']/df['cum'].iloc[-1]
x_all = (np.arange(start = 0 , stop = len(df['cum']), step = 1)/(len(df['cum'])))

y_25_75 = df['cum'].iloc[int(df.index[-1]*0.25):int(df.index[-1]*0.75)]
y_25_75 = y_25_75/max(y_25_75)
x_25_75 = np.arange(start = 0 , stop = len(y_25_75), step = 1)/(len(y_25_75))

In [45]:
print('Q3-Q1 (in pax):')
df['balance'].iloc[int(df.index[-1]*0.25):int(df.index[-1]*0.75)].describe().apply(lambda x: format(x/(10**0), 'f'))

Q3-Q1 (in pax):


count    43002.000000
mean         8.477693
std         10.402178
min          0.126041
25%          0.950017
50%          4.689466
75%         10.828251
max         45.100000
Name: balance, dtype: object

In [ ]:
print('\n\n')
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot()
plt.grid()
plt.title(r'L o r e n z \ \ C u r v e'+'\n', fontsize=50)
plt.xlabel('\n'+r'\% \ \ of \ \ A d d r e s s e s', fontsize=30)
plt.ylabel(r'\% \ o f \ \ t o t a l \ \ P A X \ \ s u p p l y'+'\n',  fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.plot(x_all,y_all, linewidth = 5, color = '#2D728F', label = r'$\ All$')
ax.plot(x_25_75,y_25_75, linewidth = 5, color = '#87AEDF', label = r'$\ Q_3 - Q_1$')
plt.legend(fontsize= 35)
plt.plot([0, 1], [0, 1], transform=ax.transAxes, linewidth = 4, ls = (0, (5, 10)), color = 'black')
ax.set_xlim([0,1.05])
plt.savefig('../pics/paxos/paxos_lorenzcurve.pdf')